In [ ]:
import duckdb

con = duckdb.connect("drilldown.duckdb")
con.execute("INSTALL excel; LOAD excel;")

sheets = ["Companies","Contacts","Products","Deals","Invoices","Invoice lines"]
for s in sheets:
    con.execute(f"""
        CREATE OR REPLACE TABLE raw_{s.lower().replace(' ','_')} AS
        SELECT * FROM read_excel('data/Drilldown_case.xlsx', sheet='{s}');
    """)
